# **Artificial Intelligence Programming Assignment Question 2**
Group Members:
- Darren Abbot ak Manggon, 11811011274
- Tuan Arif Uwais, 1181101025
- Puteri Nurul Izzah, 1181100771

## Pymaze Initialization and 5 Mazes Needed to be created and track time to plot the search the entire maze

- ensure that variable a and variable b is more or equal to 10 for the maze size.
- start and goal can be anywhere in the maze
- make sure to import the pizza into the package
- The goal will change in run-time, so the searching algorithm will change accordingly.

## Maze 1 - A-Star Search

Install pymaze to make sure that it is installed

In [ ]:
pip install pyamaze

In [1]:
from pyamaze import maze, agent, COLOR, textLabel
from queue import PriorityQueue
import time, random

#### Defining Heuristic Function (Manhattan Distance Function)

In [2]:
def heuristic(cell_1, cell_2):
    x1, y1 = cell_1
    x2, y2 = cell_2
    
    return abs(x1 - x2) + abs(y1 - y2)

#### Defining Iterative Deepening Search

In [3]:
def iterative(m):
    print("To be added soon")
    return

#### Defining the A* Search Function:

In [4]:
def aStarSearch(m, goal):
    start = (m.rows, m.cols) #Change the start nodes
    gScore = {cell: float('inf') for cell in m.grid}
    #Initialize distance of startCell from destination Cell n
    gScore[start] = 0
    
    #Total Cost to reach the goal cell combining both GScore and H Score Function
    fScore = {cell: float('inf') for cell in m.grid}
    fScore[start] = heuristic(start, goal) # Chabge the second parameter to the goal variable
    
    open = PriorityQueue()
    
    open.put((heuristic(start, goal), heuristic(start, goal), start))
    
    aPath = {} #aStar Algorithm compatible path
    searchPath = [] #Used to find spaces that have been explored
    
    #print("Starting Queue: ", open.queue)
    
    #Loop Sequence
    while not open.empty():
        currentCell = open.get()[2] #Getting coordinates in 2nd parameter
        searchPath.append(currentCell)
        
        #Replace with goal variable later on
        if currentCell == goal:
            #print("We reached the start Node")
            break
        for direction in "ESNW":
            #print("Direction: ", direction)
            if m.maze_map[currentCell][direction] == True:
                if direction == "E":
                    childCell = (currentCell[0], currentCell[1] + 1 )         
                if direction =="W":
                    childCell = (currentCell[0], currentCell[1] - 1 )
                if direction == "N":
                    childCell = (currentCell[0]-1, currentCell[1])
                if direction == "S":
                    childCell = (currentCell[0]+1, currentCell[1] )
                    
                temp_gScore = gScore[currentCell] + 1
                temp_fScore = temp_gScore + heuristic(childCell, goal) #Again, replace with goal variable
                
                if temp_fScore < fScore[childCell]:
                    #print("Founded a path: ", childCell)
                    gScore[childCell] = temp_gScore
                    fScore[childCell] = temp_fScore
                    open.put((temp_fScore, heuristic(childCell, goal), childCell))
                    aPath[childCell] = currentCell
                
                #else:
                    #print("Skipped, path has been founded before")
        #print()
            
    #Start initialize path for agent to move
    forwardPath = {}
    cell = goal #Replace with goal variable
    print(aPath)
    print(cell)
    #When the cell reaches the start Node, stop and return all list and dictionary
    while cell != start:
        forwardPath[ aPath[cell] ] = cell
        cell = aPath[cell]
        #print("Current Forward Path: ", forwardPath )
        #print("Next Cell: ", cell)
        
    #print("It adds the path for the agent from the goal node to the start node")
    return forwardPath, aPath, searchPath            
    

#### Initialize a normal maze with a bit of randomness

In [ ]:
#initialize rows, columns and loopPercentage
a = 10
b = 10
loopPercentageC = 30
goal1_x, goal1_y = 2,5

m = maze(a, b)
goal1 = goal1_x, goal1_y
goal2 = random.choice(m.grid)
goal3 = random.choice(m.grid)

m.CreateMaze(goal1_x, goal1_y,loopPercent=loopPercentageC)

#Initialze Paths and Agent
#Search Time record 1
start_timeA = time.perf_counter()
pathAStar = aStarSearch(m, goal1)
end_timeA = time.perf_counter()


#Search Time B - Got problems still
start_timeB = time.perf_counter()
pathAStar2 = aStarSearch(m, goal2)
end_timeB = time.perf_counter()


pathAStar3 = aStarSearch(m, goal3)


agentA = agent(m, footprints = True)
agentB = agent(m, shape = "arrow", footprints = True, color = COLOR.red)

agentAStarPath = agent(m, footprints = True, color=COLOR.yellow, filled = True)
agentAStarPathB = agent(m, footprints = True, color=COLOR.blue, filled = True)

#Trace the paths
m.tracePath({agentAStarPath: pathAStar[2]}, showMarked = True, delay = 500)

#Trace the agent movement

m.tracePath({agentA: pathAStar[0]})


#Trace the agent movement, i don't think this is the way to track the time taken
m.tracePath({agentAStarPathB: pathAStar2[2]}, showMarked = True, delay = 300)

m.tracePath({agentB: pathAStar2[0]}, kill = True)

#Print all here!
print()
print("Time taken for A* Search A to finish: ", end_timeA - start_timeA, " seconds")
print("Time taken for A* Search B to finish: ", end_timeB - start_timeB, " seconds")

print("Goal 1 Manually Generated: ", goal1)
print("Goal 2 Randomly Generated: ", goal2)
print("Goal 3 Randomly Generated: ", goal3)



m.run()

{(9, 10): (10, 10), (10, 9): (10, 10), (8, 10): (9, 10), (7, 10): (8, 10), (7, 9): (7, 10), (6, 9): (7, 9), (6, 10): (6, 9), (5, 9): (6, 9), (4, 9): (5, 9), (4, 8): (4, 9), (5, 8): (4, 8), (3, 8): (4, 8), (2, 8): (3, 8), (3, 7): (3, 8), (1, 8): (2, 8), (2, 7): (2, 8), (1, 7): (2, 7), (3, 6): (3, 7), (2, 6): (3, 6), (3, 5): (3, 6), (9, 9): (10, 9), (10, 8): (10, 9), (9, 8): (9, 9), (9, 7): (9, 8), (10, 7): (10, 8), (9, 6): (9, 7), (8, 6): (9, 6), (9, 5): (9, 6), (8, 7): (8, 6), (8, 5): (8, 6), (8, 4): (8, 5), (9, 4): (9, 5), (10, 6): (10, 7), (10, 5): (10, 6), (10, 4): (10, 5), (1, 6): (1, 7), (1, 5): (1, 6), (1, 4): (1, 5), (1, 9): (1, 8), (8, 3): (8, 4), (8, 8): (8, 7), (7, 7): (8, 7), (7, 8): (7, 7), (7, 6): (7, 7), (6, 6): (7, 6), (7, 5): (7, 6), (6, 5): (6, 6), (6, 4): (6, 5), (7, 4): (7, 5), (5, 10): (6, 10), (4, 10): (5, 10), (3, 10): (4, 10), (3, 9): (3, 10), (2, 9): (3, 9), (2, 10): (2, 9), (10, 3): (10, 4), (2, 4): (1, 4), (1, 3): (1, 4), (2, 5): (2, 4), (3, 4): (2, 4)}
(2, 5)

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\darre\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\darre\anaconda3\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\darre\anaconda3\lib\site-packages\pyamaze\pyamaze.py", line 739, in _tracePathSingle
    new=p[(a.x,a.y)]
KeyError: (8, 9)


In [ ]:
print(str(m.cols) + " and " + str(m.rows))

## Maze 2 - Iterative Deepening Search or Iterative Deepening A* Search